In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pretrain/pretraining/Youtube/Youtube-620.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-525.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-57.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-1091.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-1100.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-676.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-708.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-532.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-236.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-166.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-189.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-649.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-754.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-382.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-299.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-504.txt
/kaggle/input/pretrain/pretraining/Youtube/Youtube-938.

In [2]:
import pdb
import os
import numpy as np

In [3]:
ComputeInterArrival = True
DescretesizeLength = False
DirectionLengthCombined = True
NormalizeLength = True
NormalizeInterArrival = True
MaxLength = 1434
MaxInterArrival = 1
Starting_point = 0
StartingPointMultiply = 13
Num_of_extracted_subflow = 100
PaddingEnable = True
PadAtTheBegining = True
PaddingThreshold = 20


CompureStatisticsInThisScript = True

NumOfCrossValidationFolds = 1


In [4]:
np.random.seed(10)

timestep = 120
SkipPacketsForSampling = 1
IncrementalSampling = False
NumberOfSamplesUntiIncrement = 10000
IncrementalStepMultiplier = 1

In [5]:
def loadData(dirPath, class_label,  extractedFlows = 0):
    #If it is not set, use the global value
    if extractedFlows == 0:
        extractedFlows = Num_of_extracted_subflow

    pathDir = os.listdir(dirPath)

    datalist = []
    labellist = []
    FileCounter = 0
    FlowCounter = 0
    SubflowCounter = 0

    # added by Shahbaz
    custom_features = [
        # 0,    #timestamp
        1,  # RelativeTime
        2,  # length
        # 3    #Direction
    ]

    for folder, subs, files in os.walk(dirPath):

        # if FlowCounter > 1000:
        #     break

        np.random.shuffle(files)

        for file in files:
            filename = folder + "/" + file
            with open(filename) as f:
                
                FileCounter += 1
                
                EntireFile = []
                for line in f:
                    data = line.split()
                    try:
                        EntireFile.append(data)
                    except:
                        print(EntireFile)
                        pdb.set_trace()
                try:
                    EntireFile = np.array(EntireFile).astype(np.float)
                except:
                    print(EntireFile)
                    pdb.set_trace()

                FileLenght = len(EntireFile)

                if CompureStatisticsInThisScript:
                    Duration = EntireFile[-1,1]
                    Bandwidth = np.sum(EntireFile[:,2])/Duration
                    temp_label = [Bandwidth, Duration, class_label]

                SubflowFromAFile = 0

                #Skip the fist few packets in the file
                if(Starting_point!=0):
                    for jjj in range(Starting_point):
                        line = f.readline()
                        if not line:
                            break

                for subflow in range(extractedFlows):

                    startingPoint = 0
                    if subflow == 0:
                        startingPoint = 0
                    else:
                        startingPoint = np.random.randint(1, FileLenght-timestep)
                    # startingPoint = Starting_point + subflow*StartingPointMultiply
                    
                    linedata = []
                    Prev_time = 0;  #Time of the first packet in the subflow
    
                    numOfSamples = 0
                    i = startingPoint
                    SkipSamples = SkipPacketsForSampling
                    while(numOfSamples < timestep):
                        if i>=FileLenght:
                            break

                        data = list(EntireFile[i])  #To clone the list, not refering to the same list
                        
                        #shahbaz: To descretesize the the length
                        if DescretesizeLength:
                            data[2] = str(int(int(data[2])/100))

                        if DirectionLengthCombined:
                            if data[3]=="0":
                                if float(data[2])>0:
                                    data[2] = str(-1 * float(data[2]))
                                    
                        if NormalizeLength:
                            data[2] = str(float(data[2])/MaxLength)
                            
                        if ComputeInterArrival:
                            if i==startingPoint:
                                Prev_time = float(data[1])
                                data[1] = str(0)
                            else:
                                temporary = str(float(data[1]) - Prev_time)
                                Prev_time = float(data[1])
                                data[1] = temporary
                        if NormalizeInterArrival:
                            ttt = float(data[1]) / MaxInterArrival
                            if ttt > 1:
                                ttt=1
                            data[1]=(ttt-0.5)*2
                            

                        try:
                            data2 = [float(data[j]) for j in custom_features]
                        except (IndexError, ValueError) as e:
                            pass
                            print("Couldn't retrieve all data",filename)
                        else:
                            linedata += data2
                
                        numOfSamples += 1
                        i += SkipSamples
                        if IncrementalSampling:
                            if numOfSamples % NumberOfSamplesUntiIncrement == 0:
                                SkipSamples = int(SkipSamples*IncrementalStepMultiplier)

    
                    if (len(linedata) < len(custom_features) * timestep):
                        if (PaddingThreshold > len(linedata)/len(custom_features) ):
                            continue
                        #print(linedata)
                        if (PaddingEnable):
                            while(len(linedata) < len(custom_features) * timestep):
                                pad = []
                                pad.extend(np.ones(len(custom_features)) * 0)
                                if PadAtTheBegining:
                                    pad.extend(linedata)
                                    linedata = pad
                                else:
                                    linedata.extend(pad)
                            #print(linedata)
                        else:
                            continue
                    np.nan_to_num(linedata)
                    datalist.append(linedata)

                    SubflowCounter+=1
                    SubflowFromAFile+=1

                total_labels = [temp_label] * SubflowFromAFile
                labellist.extend(total_labels)
                FlowCounter+=1
                print(filename,temp_label)
    ratio = SubflowCounter/FlowCounter
    print(dirPath + ":" + str(FlowCounter) + "/" + str(len(pathDir)) + " - Subflows:" + str(SubflowCounter) + " - Ratio:", str(ratio))
    return (np.array(datalist), np.array(labellist))


In [6]:
BaseDirectory = "../input/pretrain/pretraining"
(data1, label1) = loadData(BaseDirectory + "/Google Drive", 1, extractedFlows=1)
(data2, label2) = loadData(BaseDirectory + "/Youtube", 2,  extractedFlows=1)
(data3, label3) = loadData(BaseDirectory + "/Google Doc", 3,  extractedFlows=1)
(data4, label4) = loadData(BaseDirectory + "/Google Search", 4, extractedFlows=1)
(data5, label5) = loadData(BaseDirectory + "/Google Music", 5, extractedFlows=1)



../input/pretrain/pretraining/Google Drive/GoogleDrive-596.txt [174036.72599118346, 59.0481, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-920.txt [724907.3809963254, 11.9198, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-544.txt [1040181.451774775, 12.4551, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-910.txt [450444.85980042594, 29.5329, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-526.txt [159735.18251692966, 60.5889, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-1062.txt [230049.21344507148, 38.3635, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-865.txt [166640.9779101514, 64.464, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-1015.txt [407834.20373152837, 27.4204, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-1530.txt [179280.7287151805, 64.4861, 1]
../input/pretrain/pretraining/Google Drive/GoogleDrive-1106.txt [1001445.7648814258, 13.2027, 1]
../input/pretrain/pretraining/Google Driv

In [7]:
test_size = 30
val_size = 30
train1 = data1[:-(test_size+val_size)]
train2 = data2[:-(test_size+val_size)]
train3 = data3[:-(test_size+val_size)]
train4 = data4[:-(test_size+val_size)]
train5 = data5[:-(test_size+val_size)]
val1 = data1[-(test_size+val_size):-test_size]
val2 = data2[-(test_size+val_size):-test_size]
val3 = data3[-(test_size+val_size):-test_size]
val4 = data4[-(test_size+val_size):-test_size]
val5 = data5[-(test_size+val_size):-test_size]
test1 = data1[-test_size:]
test2 = data2[-test_size:]
test3 = data3[-test_size:]
test4 = data4[-test_size:]
test5 = data5[-test_size:]

In [8]:
trainL1 = label1[:-(test_size+val_size)]
trainL2 = label2[:-(test_size+val_size)]
trainL3 = label3[:-(test_size+val_size)]
trainL4 = label4[:-(test_size+val_size)]
trainL5 = label5[:-(test_size+val_size)]
valL1 = label1[-(test_size+val_size):-test_size]
valL2 = label2[-(test_size+val_size):-test_size]
valL3 = label3[-(test_size+val_size):-test_size]
valL4 = label4[-(test_size+val_size):-test_size]
valL5 = label5[-(test_size+val_size):-test_size]
testL1 = label1[-test_size:]
testL2 = label2[-test_size:]
testL3 = label3[-test_size:]
testL4 = label4[-test_size:]
testL5 = label5[-test_size:]

In [9]:
train_data = np.concatenate((train1, train2, train3, train4, train5), axis=0)
val_data = np.concatenate((val1, val2, val3, val4, val5), axis=0)
test_data = np.concatenate((test1, test2, test3, test4, test5), axis=0)

train_label = np.concatenate((trainL1, trainL2, trainL3, trainL4, trainL5), axis=0)
val_label = np.concatenate((valL1, valL2, valL3, valL4, valL5), axis=0)
test_label = np.concatenate((testL1, testL2, testL3, testL4, testL5), axis=0)

In [10]:
np.save("trainData.npy", train_data)
np.save("trainLabel.npy", train_label)

np.save("valData.npy", val_data)
np.save("valLabel.npy", val_label)

np.save("testData.npy", test_data)
np.save("testLabel.npy", test_label)

print(train_data.shape, train_label.shape)
print(val_data.shape, val_label.shape)
print(test_data.shape, test_label.shape)

(6139, 240) (6139, 3)
(150, 240) (150, 3)
(150, 240) (150, 3)


# Single Task

In [11]:
import numpy as np
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.optimizers import Adam # - Works
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Activation
#from keras.optimizers import Adam

In [12]:
timestep = 60
np.random.seed(10)

num_class = 5
train_sample_per_class = 10

# For Bandwith Prediction:
# PredictionType = 0
# TrainWithAllData = True

#For Duration Prediction:
# PredictionType = 1
# num_class = 4
# TrainWithAllData = True

#For Traffic Class Prediction:
PredictionType = 2
TrainWithAllData = False



In [13]:
trainData = np.load("./trainData.npy")
trainlabel = np.load("./trainLabel.npy")
trainData = trainData[:, -timestep*2:]
trainlabel = trainlabel[:, -timestep*2:]
trainlabel = trainlabel.astype(int)

valData = np.load("./valData.npy")
valLabel = np.load("./valLabel.npy")
valData = valData[:, -timestep*2:]
valLabel = valLabel[:, -timestep*2:]
valLabel = valLabel.astype(int)

testData = np.load("./testData.npy")
testLabel = np.load("./testLabel.npy")
testData = testData[:, -timestep*2:]
testLabel = testLabel[:, -timestep*2:]
testLabel = testLabel.astype(int)

In [14]:
for i in range(trainlabel.shape[0]):
    #Categorizing Bandwidth
    if trainlabel[i, 0] < 10000:
        trainlabel[i, 0] = 1
    elif trainlabel[i, 0] < 50000:
        trainlabel[i, 0] = 2
    elif trainlabel[i, 0] < 100000:
        trainlabel[i, 0] = 3
    elif trainlabel[i, 0] < 1000000:
        trainlabel[i, 0] = 4
    else:
        trainlabel[i, 0] = 5
    #Categorizing Duration
    if trainlabel[i, 1] < 10:
        trainlabel[i, 1] = 1
    elif trainlabel[i, 1] < 30:
        trainlabel[i, 1] = 2
    elif trainlabel[i, 1] < 60:
        trainlabel[i, 1] = 3
    else:
        trainlabel[i, 1] = 4

In [15]:
for i in range(valLabel.shape[0]):
    #Categorizing Bandwidth
    if valLabel[i, 0] < 10000:
        valLabel[i, 0] = 1
    elif valLabel[i, 0] < 50000:
        valLabel[i, 0] = 2
    elif valLabel[i, 0] < 100000:
        valLabel[i, 0] = 3
    elif valLabel[i, 0] < 1000000:
        valLabel[i, 0] = 4
    else:
        valLabel[i, 0] = 5
    #Categorizing Duration
    if valLabel[i, 1] < 10:
        valLabel[i, 1] = 1
    elif valLabel[i, 1] < 30:
        valLabel[i, 1] = 2
    elif valLabel[i, 1] < 60:
        valLabel[i, 1] = 3
    else:
        valLabel[i, 1] = 4

In [16]:
for i in range(testLabel.shape[0]):
    #Categorizing Bandwidth
    if testLabel[i, 0] < 10000:
        testLabel[i, 0] = 1
    elif testLabel[i, 0] < 50000:
        testLabel[i, 0] = 2
    elif testLabel[i, 0] < 100000:
        testLabel[i, 0] = 3
    elif testLabel[i, 0] < 1000000:
        testLabel[i, 0] = 4
    else:
        testLabel[i, 0] = 5
    #Categorizing Duration
    if testLabel[i, 1] < 10:
        testLabel[i, 1] = 1
    elif testLabel[i, 1] < 30:
        testLabel[i, 1] = 2
    elif testLabel[i, 1] < 60:
        testLabel[i, 1] = 3
    else:
        testLabel[i, 1] = 4

In [17]:
trainlabel = trainlabel[:, PredictionType]
testLabel = testLabel[:, PredictionType]
valLabel = valLabel[:, PredictionType]

In [18]:
train_size = trainlabel.shape[0]
Y_train = np.zeros((train_size, num_class))
Y_train[np.arange(train_size),trainlabel[:]-1] = 1

val_size = valLabel.shape[0]
Y_val = np.zeros((val_size, num_class))
Y_val[np.arange(val_size),valLabel[:]-1] = 1

test_size = testLabel.shape[0]
Y_test = np.zeros((test_size, num_class))
Y_test[np.arange(test_size),testLabel[:]-1] = 1

trainData = trainData.reshape((trainData.shape[0], timestep, 2))
valData = valData.reshape((valData.shape[0], timestep, 2))
testData = testData.reshape((testData.shape[0], timestep, 2))

In [19]:
if not TrainWithAllData:
    class_counter = np.zeros((num_class))
    x_train = np.zeros((num_class*train_sample_per_class, timestep, 2))
    y_train = np.zeros((num_class*train_sample_per_class, 5))
    j = 0
    for i in range(train_size):
        class_id = trainlabel[i] - 1
        if class_counter[class_id] < train_sample_per_class:
            x_train[j] = trainData[i]
            y_train[j] = Y_train[i]
            j += 1
            class_counter[class_id] += 1

    trainData = x_train
    Y_train = y_train

In [20]:
def base_model():

    model_input = Input(shape=(timestep,2))

    x = Conv1D(32, 3, activation='relu')(model_input)
    x = Conv1D(32, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(64, 3, activation='relu')(x)
    x = Conv1D(64, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(128, 3, activation='relu')(x)
    x = Conv1D(128, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Flatten()(x)

    x3 = Dense(256)(x)
    x3 = Activation('relu')(x3)
    x3 = Dense(256)(x3)
    x3 = Activation('relu')(x3)
    output3 = Dense(num_class, activation='softmax', name='Class')(x3)

    model = Model(inputs=model_input, outputs=[output3])
    opt = Adam(clipnorm = 1.)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [21]:
model = base_model()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [22]:
model.fit(trainData, Y_train, validation_data = (valData, Y_val),
          batch_size = 64, epochs = 30, verbose = True, shuffle = True)


2021-11-25 10:08:13.206665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
1/1 [==============================] - 3s 3s/step - loss: 1.6111 - accuracy: 0.2000 - val_loss: 1.6048 - val_accuracy: 0.2000
Epoch 2/30
1/1 [==============================] - 0s 80ms/step - loss: 1.6028 - accuracy: 0.2000 - val_loss: 1.5984 - val_accuracy: 0.2000
Epoch 3/30
1/1 [==============================] - 0s 81ms/step - loss: 1.5948 - accuracy: 0.2000 - val_loss: 1.5905 - val_accuracy: 0.2000
Epoch 4/30
1/1 [==============================] - 0s 85ms/step - loss: 1.5840 - accuracy: 0.2000 - val_loss: 1.5789 - val_accuracy: 0.2000
Epoch 5/30
1/1 [==============================] - 0s 85ms/step - loss: 1.5690 - accuracy: 0.2200 - val_loss: 1.5619 - val_accuracy: 0.2400
Epoch 6/30
1/1 [==============================] - 0s 86ms/step - loss: 1.5474 - accuracy: 0.3400 - val_loss: 1.5380 - val_accuracy: 0.3267
Epoch 7/30
1/1 [==============================] - 0s 86ms/step - loss: 1.5181 - accuracy: 0.4000 - val_loss: 1.5058 - val_accuracy: 0.3133
Epoch 8/30
1/1 [=============

In [23]:
result = model.evaluate([testData], [Y_test])
print(result)

5/5 [==============================] - 0s 10ms/step - loss: 0.9364 - accuracy: 0.6133
[0.9363958835601807, 0.6133333444595337]


# transferlearning

In [24]:
timestep = 60
np.random.seed(10)

num_class = 5
retrain_sample_per_class = 10
pretrain_test_sample_per_class = 10

In [25]:
trainData = np.load("./trainData.npy")
trainlabel = np.load("./trainLabel.npy")
trainData = trainData[:, -timestep*2:]
trainlabel = trainlabel[:, -timestep*2:]
trainlabel = trainlabel.astype(int)

testData = np.load("./testData.npy")
testLabel = np.load("./testLabel.npy")
testData = testData[:, -timestep*2:]
testLabel = testLabel[:, -timestep*2:]
testLabel = testLabel.astype(int)

In [26]:
for i in range(trainlabel.shape[0]):
    #Categorizing Bandwidth
    if trainlabel[i, 0] < 10000:
        trainlabel[i, 0] = 1
    elif trainlabel[i, 0] < 50000:
        trainlabel[i, 0] = 2
    elif trainlabel[i, 0] < 100000:
        trainlabel[i, 0] = 3
    elif trainlabel[i, 0] < 1000000:
        trainlabel[i, 0] = 4
    else:
        trainlabel[i, 0] = 5
    #Categorizing Duration
    if trainlabel[i, 1] < 10:
        trainlabel[i, 1] = 1
    elif trainlabel[i, 1] < 30:
        trainlabel[i, 1] = 2
    elif trainlabel[i, 1] < 60:
        trainlabel[i, 1] = 3
    else:
        trainlabel[i, 1] = 4

In [27]:
for i in range(testLabel.shape[0]):
    #Categorizing Bandwidth
    if testLabel[i, 0] < 10000:
        testLabel[i, 0] = 1
    elif testLabel[i, 0] < 50000:
        testLabel[i, 0] = 2
    elif testLabel[i, 0] < 100000:
        testLabel[i, 0] = 3
    elif testLabel[i, 0] < 1000000:
        testLabel[i, 0] = 4
    else:
        testLabel[i, 0] = 5
    #Categorizing Duration
    if testLabel[i, 1] < 10:
        testLabel[i, 1] = 1
    elif testLabel[i, 1] < 30:
        testLabel[i, 1] = 2
    elif testLabel[i, 1] < 60:
        testLabel[i, 1] = 3
    else:
        testLabel[i, 1] = 4


In [28]:
trainmask = np.zeros((trainlabel.shape[0],256))

retrain_size = num_class*retrain_sample_per_class
pretrain_test_size = num_class*pretrain_test_sample_per_class
train_size = trainlabel.shape[0]
pretrain_size = train_size - retrain_size - pretrain_test_size

In [29]:
x_pretrain = np.zeros((pretrain_size, timestep*2))
y_pretrain = np.zeros((pretrain_size, 2))

x_test_pretrain = np.zeros((pretrain_test_size, timestep*2))
y_test_pretrain = np.zeros((pretrain_test_size, 2))

x_retrain = np.zeros((retrain_size, timestep*2))
y_retrain = np.zeros((retrain_size))


In [30]:
class_counter = np.zeros((num_class))
j = 0   #Counter for pretrain train set
l = 0   #counter for pretrain test set
k = 0   #counter for retrain train set
for i in range(train_size):
    class_id = trainlabel[i,2] - 1
    if class_counter[class_id] < retrain_sample_per_class:
        x_retrain[k] = trainData[i]
        y_retrain[k] = trainlabel[i, 2]
        k += 1
        class_counter[class_id] += 1
    elif class_counter[class_id] < retrain_sample_per_class + pretrain_test_sample_per_class:
        x_test_pretrain[l] = trainData[i]
        y_test_pretrain[l] = trainlabel[i, 0:2]
        l += 1
        class_counter[class_id] += 1
    else:
        x_pretrain[j] = trainData[i]
        y_pretrain[j] = trainlabel[i, 0:2]
        j += 1
        class_counter[class_id] += 1

In [31]:
y_pretrain = y_pretrain.astype(int)
y_test_pretrain = y_test_pretrain.astype(int)
y_retrain = y_retrain.astype(int)


In [32]:
Y_pretrain1 = np.zeros((pretrain_size, 5))
Y_pretrain1[np.arange(pretrain_size), y_pretrain[:, 0]-1] = 1
Y_pretrain2 = np.zeros((pretrain_size, 4))
Y_pretrain2[np.arange(pretrain_size), y_pretrain[:, 1]-1] = 1

Y_test_pretrain1 = np.zeros((pretrain_test_size, 5))
Y_test_pretrain1[np.arange(pretrain_test_size), y_test_pretrain[:, 0]-1] = 1
Y_test_pretrain2 = np.zeros((pretrain_test_size, 4))
Y_test_pretrain2[np.arange(pretrain_test_size), y_test_pretrain[:, 1]-1] = 1

Y_retrain = np.zeros((retrain_size, 5))
Y_retrain[np.arange(retrain_size), y_retrain[:]-1] = 1

retrain_test_size = testLabel.shape[0]
Y_test_retrain = np.zeros((retrain_test_size,5))
Y_test_retrain[np.arange(retrain_test_size), testLabel[:,2]-1] = 1

pretrainData = x_pretrain.reshape((x_pretrain.shape[0], timestep, 2))
pretrainTestData = x_test_pretrain.reshape((x_test_pretrain.shape[0], timestep, 2))
retrainData = x_retrain.reshape((x_retrain.shape[0], timestep, 2))
retrainTestData = testData.reshape((testData.shape[0], timestep, 2))

In [33]:
def base_model():

    model_input = Input(shape=(timestep, 2))

    x = Conv1D(32, 3, activation='relu')(model_input)
    x = Conv1D(32, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(64, 3, activation='relu')(x)
    x = Conv1D(64, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(128, 3, activation='relu')(x)
    x = Conv1D(128, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Flatten()(x)

    x1 = Dense(256)(x)
    x1 = Activation('relu')(x1)
    x1 = Dense(256, name='feature_extractor')(x1)
    x1 = Activation('relu')(x1)
    output1 = Dense(5, activation='softmax', name='Bandwidth')(x1)
    output2 = Dense(4, activation='softmax', name='Duration')(x1)

    model = Model(inputs=model_input, outputs=[output1, output2])
    opt = Adam(clipnorm = 1.)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opt, metrics=['accuracy'])

    return model

In [34]:
model = base_model()

model.fit(pretrainData, [Y_pretrain1, Y_pretrain2],
          validation_data = (pretrainTestData, [Y_test_pretrain1, Y_test_pretrain2]),
          batch_size = 64, epochs = 20, verbose = True, shuffle = True)

Epoch 1/20
95/95 [==============================] - 5s 32ms/step - loss: 1.9873 - Bandwidth_loss: 0.9126 - Duration_loss: 1.0747 - Bandwidth_accuracy: 0.6279 - Duration_accuracy: 0.4968 - val_loss: 1.4879 - val_Bandwidth_loss: 0.5602 - val_Duration_loss: 0.9278 - val_Bandwidth_accuracy: 0.7200 - val_Duration_accuracy: 0.4800
Epoch 2/20
95/95 [==============================] - 3s 27ms/step - loss: 1.4308 - Bandwidth_loss: 0.5668 - Duration_loss: 0.8641 - Bandwidth_accuracy: 0.7607 - Duration_accuracy: 0.5936 - val_loss: 1.3125 - val_Bandwidth_loss: 0.4533 - val_Duration_loss: 0.8592 - val_Bandwidth_accuracy: 0.7400 - val_Duration_accuracy: 0.6000
Epoch 3/20
95/95 [==============================] - 3s 27ms/step - loss: 1.2814 - Bandwidth_loss: 0.4852 - Duration_loss: 0.7962 - Bandwidth_accuracy: 0.7887 - Duration_accuracy: 0.6314 - val_loss: 1.2658 - val_Bandwidth_loss: 0.4436 - val_Duration_loss: 0.8222 - val_Bandwidth_accuracy: 0.7400 - val_Duration_accuracy: 0.6000
Epoch 4/20
95/95 [=

In [35]:
def target_model(source_model):
    feature_extractor_layer = source_model.get_layer('feature_extractor').output
    output = Dense(5, activation='softmax', name='TrafficClass')(feature_extractor_layer)

    target_model = Model(inputs=source_model.input, outputs=output)
    opt = Adam(clipnorm = 1.)
    target_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return target_model

In [36]:
new_model = target_model(model)



In [37]:
for layer in new_model.layers[:10]:
    layer.trainable = False
for layer in new_model.layers[10:]:
    layer.trainable = True

In [38]:
new_model.fit(retrainData, Y_retrain, validation_data = (retrainTestData, Y_test_retrain),
          batch_size=64, epochs=20, verbose=True, shuffle=True)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 1.6496 - accuracy: 0.2600 - val_loss: 1.1849 - val_accuracy: 0.5067
Epoch 2/20
1/1 [==============================] - 0s 96ms/step - loss: 1.1275 - accuracy: 0.4800 - val_loss: 0.9431 - val_accuracy: 0.6533
Epoch 3/20
1/1 [==============================] - 0s 88ms/step - loss: 0.8269 - accuracy: 0.7600 - val_loss: 0.7459 - val_accuracy: 0.8000
Epoch 4/20
1/1 [==============================] - 0s 92ms/step - loss: 0.5969 - accuracy: 0.8800 - val_loss: 0.5903 - val_accuracy: 0.8400
Epoch 5/20
1/1 [==============================] - 0s 92ms/step - loss: 0.4279 - accuracy: 0.8800 - val_loss: 0.4778 - val_accuracy: 0.8333
Epoch 6/20
1/1 [==============================] - 0s 86ms/step - loss: 0.3115 - accuracy: 0.8800 - val_loss: 0.3996 - val_accuracy: 0.8533
Epoch 7/20
1/1 [==============================] - 0s 88ms/step - loss: 0.2323 - accuracy: 0.9200 - val_loss: 0.3481 - val_accuracy: 0.8800
Epoch 8/20
1/1 [=============

# multitaskMasked

In [39]:
timestep = 60
np.random.seed(10)

num_class = 5
train_sample_per_class = 20
lambda_value = 1

In [40]:
trainData = np.load("./trainData.npy")
trainlabel = np.load("./trainLabel.npy")
# trainData = trainData[:, -timestep*2:]
# trainlabel = trainlabel[:, -timestep*2:]
trainData = trainData[:, :timestep*2]
trainlabel = trainlabel[:, :timestep*2]
trainlabel = trainlabel.astype(int)

trainmask = np.zeros((trainlabel.shape[0],256))

In [41]:
class_counter = np.zeros((num_class))
train_size = trainlabel.shape[0]
j = 0
for i in range(train_size):
    class_id = trainlabel[i,2] - 1
    if class_counter[class_id] < train_sample_per_class:
        trainmask[i, :] = 1
        j += 1
        class_counter[class_id] += 1
print("unmasked samples: ", str(np.sum(trainmask==1)/256))

unmasked samples:  100.0


In [42]:
valData = np.load("./valData.npy")
valLabel = np.load("./valLabel.npy")
# testData = testData[:, -timestep*2:]
# testLabel = testLabel[:, -timestep*2:]
valData = valData[:, :timestep*2]
valLabel = valLabel[:, :timestep*2]

valLabel = valLabel.astype(int)
valmask = np.ones((valLabel.shape[0], 256))
valmask[:,:]=1

In [43]:
testData = np.load("./testData.npy")
testLabel = np.load("./testLabel.npy")
# testData = testData[:, -timestep*2:]
# testLabel = testLabel[:, -timestep*2:]
testData = testData[:, :timestep*2]
testLabel = testLabel[:, :timestep*2]

testLabel = testLabel.astype(int)
testmask = np.ones((testLabel.shape[0], 256))
testmask[:,:]=1


In [44]:
for i in range(trainlabel.shape[0]):
    #Categorizing Bandwidth
    if trainlabel[i, 0] < 10000:
        trainlabel[i, 0] = 1
    elif trainlabel[i, 0] < 50000:
        trainlabel[i, 0] = 2
    elif trainlabel[i, 0] < 100000:
        trainlabel[i, 0] = 3
    elif trainlabel[i, 0] < 1000000:
        trainlabel[i, 0] = 4
    else:
        trainlabel[i, 0] = 5
    #Categorizing Duration
    if trainlabel[i, 1] < 10:
        trainlabel[i, 1] = 1
    elif trainlabel[i, 1] < 30:
        trainlabel[i, 1] = 2
    elif trainlabel[i, 1] < 60:
        trainlabel[i, 1] = 3
    else:
        trainlabel[i, 1] = 4

In [45]:
for i in range(valLabel.shape[0]):
    #Categorizing Bandwidth
    if valLabel[i, 0] < 10000:
        valLabel[i, 0] = 1
    elif valLabel[i, 0] < 50000:
        valLabel[i, 0] = 2
    elif valLabel[i, 0] < 100000:
        valLabel[i, 0] = 3
    elif valLabel[i, 0] < 1000000:
        valLabel[i, 0] = 4
    else:
        valLabel[i, 0] = 5
    #Categorizing Duration
    if valLabel[i, 1] < 10:
        valLabel[i, 1] = 1
    elif valLabel[i, 1] < 30:
        valLabel[i, 1] = 2
    elif valLabel[i, 1] < 60:
        valLabel[i, 1] = 3
    else:
        valLabel[i, 1] = 4

In [46]:
for i in range(testLabel.shape[0]):
    #Categorizing Bandwidth
    if testLabel[i, 0] < 10000:
        testLabel[i, 0] = 1
    elif testLabel[i, 0] < 50000:
        testLabel[i, 0] = 2
    elif testLabel[i, 0] < 100000:
        testLabel[i, 0] = 3
    elif testLabel[i, 0] < 1000000:
        testLabel[i, 0] = 4
    else:
        testLabel[i, 0] = 5
    #Categorizing Duration
    if testLabel[i, 1] < 10:
        testLabel[i, 1] = 1
    elif testLabel[i, 1] < 30:
        testLabel[i, 1] = 2
    elif testLabel[i, 1] < 60:
        testLabel[i, 1] = 3
    else:
        testLabel[i, 1] = 4

In [47]:
train_size = trainlabel.shape[0]
Y_train1 = np.zeros((train_size,5))
Y_train1[np.arange(train_size),trainlabel[:,0]-1] = 1
Y_train2 = np.zeros((train_size,4))
Y_train2[np.arange(train_size),trainlabel[:,1]-1] = 1
Y_train3 = np.zeros((train_size,5))
Y_train3[np.arange(train_size),trainlabel[:,2]-1] = 1

val_size = valLabel.shape[0]
Y_val1 = np.zeros((val_size,5))
Y_val1[np.arange(val_size),valLabel[:,0]-1] = 1
Y_val2 = np.zeros((val_size,4))
Y_val2[np.arange(val_size),valLabel[:,1]-1] = 1
Y_val3 = np.zeros((val_size,5))
Y_val3[np.arange(val_size),valLabel[:,2]-1] = 1

test_size = testLabel.shape[0]
Y_test1 = np.zeros((test_size,5))
Y_test1[np.arange(test_size),testLabel[:,0]-1] = 1
Y_test2 = np.zeros((test_size,4))
Y_test2[np.arange(test_size),testLabel[:,1]-1] = 1
Y_test3 = np.zeros((test_size,5))
Y_test3[np.arange(test_size),testLabel[:,2]-1] = 1

# trainData = np.expand_dims(trainData, axis=-1)
# testData = np.expand_dims(testData, axis=-1)
trainData = trainData.reshape((trainData.shape[0], timestep, 2))
testData = testData.reshape((testData.shape[0], timestep, 2))
valData = valData.reshape((valData.shape[0], timestep, 2))


In [48]:
from keras.layers import multiply

In [49]:
def base_model():

    model_input = Input(shape=(timestep,2))
    mask_input = Input(shape=(256,))

    x = Conv1D(32, 3, activation='relu')(model_input)
    x = Conv1D(32, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(64, 3, activation='relu')(x)
    x = Conv1D(64, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Conv1D(128, 3, activation='relu')(x)
    x = Conv1D(128, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=(2))(x)

    x = Flatten()(x)

    x = Dense(256)(x)
    x = Activation('relu')(x)
    x = Dense(256)(x)
    x = Activation('relu')(x)

    output1 = Dense(5, activation='softmax', name='Bandwidth')(x)

    output2 = Dense(4, activation='softmax', name='Duration')(x)

    x3 = multiply([x,mask_input])
    output3 = Dense(5, activation='softmax', name='Class')(x3)

    model = Model(inputs=[model_input,mask_input], outputs=[output1, output2, output3])
    opt = Adam(clipnorm = 1.)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1,1,lambda_value], optimizer=opt, metrics=['accuracy'])

    return model

In [50]:
model = base_model()

In [51]:
model.fit([trainData,trainmask], [Y_train1, Y_train2, Y_train3],
          validation_data = ([valData, valmask], [Y_val1, Y_val2, Y_val3]),
          batch_size = 64, epochs = 20, verbose = True, shuffle = True)



Epoch 1/20
96/96 [==============================] - 6s 38ms/step - loss: 3.5957 - Bandwidth_loss: 0.9719 - Duration_loss: 1.0247 - Class_loss: 1.5992 - Bandwidth_accuracy: 0.6037 - Duration_accuracy: 0.5476 - Class_accuracy: 0.3009 - val_loss: 3.1534 - val_Bandwidth_loss: 0.8460 - val_Duration_loss: 1.0675 - val_Class_loss: 1.2398 - val_Bandwidth_accuracy: 0.6333 - val_Duration_accuracy: 0.6000 - val_Class_accuracy: 0.3733
Epoch 2/20
96/96 [==============================] - 3s 28ms/step - loss: 2.8130 - Bandwidth_loss: 0.5737 - Duration_loss: 0.6699 - Class_loss: 1.5695 - Bandwidth_accuracy: 0.7648 - Duration_accuracy: 0.7171 - Class_accuracy: 0.3092 - val_loss: 2.2692 - val_Bandwidth_loss: 0.6289 - val_Duration_loss: 0.7854 - val_Class_loss: 0.8549 - val_Bandwidth_accuracy: 0.7333 - val_Duration_accuracy: 0.7467 - val_Class_accuracy: 0.6133
Epoch 3/20
96/96 [==============================] - 3s 30ms/step - loss: 2.6141 - Bandwidth_loss: 0.4910 - Duration_loss: 0.5723 - Class_loss: 1.5

In [52]:
result = model.evaluate([testData, testmask], [Y_test1, Y_test2, Y_test3])
print(result)

5/5 [==============================] - 0s 8ms/step - loss: 0.9088 - Bandwidth_loss: 0.2560 - Duration_loss: 0.4990 - Class_loss: 0.1537 - Bandwidth_accuracy: 0.9000 - Duration_accuracy: 0.7333 - Class_accuracy: 0.9467
[0.9087796211242676, 0.2560138404369354, 0.4990401566028595, 0.1537254899740219, 0.8999999761581421, 0.7333333492279053, 0.9466666579246521]
